In [1]:
import pandas as pd

import spacy

nlp = spacy.load('en_core_web_sm')

In [2]:
similes = pd.read_table('similes.txt', header=None, names=['simile'])

%time similes['nlp'] = similes['simile'].apply(lambda text: nlp(text))

CPU times: user 24.3 s, sys: 753 ms, total: 25.1 s
Wall time: 22.2 s


In [3]:
def make_list(inp):    
    return list(map(int, inp.split()))

In [14]:
output = []

sentences = pd.read_json('similes.json', orient='records')
samples = similes[~similes['simile'].isin(sentences['original'])]

print(f'Max Samples: {len(samples)}')

count = int(input('How many samples to label?'))
print()

samples = samples.sample(count)

for ix, sample in samples.iterrows():
    
    tokens = [tk.text for tk in sample['nlp']]
    
    tks = ''
    for i, tk in enumerate(tokens):
        tks += f'{i}_{tk} '
        
    print(tks)
    
    res = {'original': sample['simile']}
    
    topic = input('Topic tokens?')
    if topic == '-':
        print()
        continue
    res['topic'] = make_list(topic)
    
    event = input('Event tokens?')
    if event == '-':
        print()
        continue
    res['event'] = make_list(event)
    
    ground = input('Ground tokens?')
    if ground == '-':
        print()
        continue
    res['ground'] = make_list(ground)
    
    vehicle = input('Vehicle tokens?')
    if vehicle == '-':
        print()
        continue
    res['vehicle'] = make_list(vehicle)

    output.append(res)
    print()
    
output = pd.DataFrame(output)

Max Samples: 776
How many samples to label?28

0_Wrangle 1_over 2_details 3_like 4_a 5_grasping 6_pawnbroker 
Topic tokens?
Event tokens?0 1 2
Ground tokens?
Vehicle tokens?4 5 6

0_Drowned 1_like 2_rats 
Topic tokens?
Event tokens?0
Ground tokens?
Vehicle tokens?2

0_Streamed 1_over 2_his 3_memory 4_like 5_a 6_forest 7_flame 
Topic tokens?
Event tokens?0 1 2 3
Ground tokens?
Vehicle tokens?5 6 7

0_He 1_was 2_as 3_splendidly 4_serious 5_as 6_a 7_reformer 
Topic tokens?0
Event tokens?1
Ground tokens?3 4
Vehicle tokens?6 7

0_Love 1_that 2_sings 3_and 4_has 5_wings 6_as 7_a 8_bird 
Topic tokens?0
Event tokens?2 3 4 5
Ground tokens?
Vehicle tokens?7 8

0_White 1_as 2_dove 3_or 4_lily 5_, 6_or 7_spirit 8_of 9_the 10_light 
Topic tokens?
Event tokens?
Ground tokens?0
Vehicle tokens?2

0_Her 1_expression 2_changed 3_with 4_the 5_rapidity 6_of 7_a 8_kaleidoscope 
Topic tokens?-

0_Whose 1_bodies 2_are 3_as 4_strong 5_as 6_alabaster 
Topic tokens?0 1
Event tokens?2
Ground tokens?4
Vehicle tok

In [15]:
output

,event,ground,original,topic,vehicle
0,"[0, 1, 2]",[],Wrangle over details like a grasping pawnbroker,[],"[4, 5, 6]"
1,[0],[],Drowned like rats,[],[2]
2,"[0, 1, 2, 3]",[],Streamed over his memory like a forest flame,[],"[5, 6, 7]"
3,[1],"[3, 4]",He was as splendidly serious as a reformer,[0],"[6, 7]"
4,"[2, 3, 4, 5]",[],Love that sings and has wings as a bird,[0],"[7, 8]"
5,[],[0],"White as dove or lily, or spirit of the light",[],[2]
6,[2],[4],Whose bodies are as strong as alabaster,"[0, 1]",[6]
7,"[0, 1, 2, 3, 4, 5]",[],"Moving in the same dull round, like blind hors...",[],"[8, 9, 10, 11, 12]"
8,[],[0],Shapeless as a sack of wool,[],"[2, 3, 4, 5]"
9,"[3, 4, 5, 6]",[],A little breeze ran through the corn like a sw...,"[0, 1, 2]","[8, 9, 10]"


In [16]:
new = pd.concat([sentences, output], axis=0)

new.to_json('similes.json', orient='records')